In [1]:
import numpy as np
rng = np.random.default_rng(0)

from glmnet.coxdev import right_censored
import rpy2
%load_ext rpy2.ipython

n = 20
event_time = rng.standard_exponential(n)
status = rng.choice([0, 1], size=n)
eta = rng.standard_normal(n)

In [2]:
rc = right_censored(event_time, status)
rc(eta)

CoxDevianceResult(deviance=100.97527029683503, grad=11   -0.187311
3    -0.148891
2    -0.114290
13    1.601763
18    0.586289
17   -0.084952
4    -0.459015
6     1.765506
0    -0.194274
7     1.458237
15    1.486070
1    -0.643340
14    0.845844
19   -1.668823
5     0.421839
12   -4.759320
8    -3.103675
16   -7.041327
10    1.414909
9    -9.175239
dtype: float64, diag_hessian=11   -0.084884
3    -0.071668
2    -0.056053
13   -0.189171
18   -0.606441
17   -0.858728
4    -0.221649
6    -0.115425
0    -0.096018
7    -0.263029
15   -0.250530
1    -0.312416
14   -0.549549
19   -1.575367
5    -0.743599
12   -1.984895
8    -1.385445
16   -2.653738
10   -0.285168
9    -2.773370
Name: sample_weight, dtype: float64)

In [17]:
%%R -i event_time,status,eta
library(glmnet)
n = length(event_time)
library(survival)
y = data.frame(time=event_time, status=status)
y_s = Surv(event_time, status)
w = rep(1, n)
print(glmnet:::coxgrad2(eta, y_s, w, diag.hessian=TRUE))
eta = as.numeric(eta)
glmnet:::coxnet.deviance2(pred=eta, y=y_s)

 [1] -0.002187567 -0.008323869  0.000000000  0.000000000 -0.004945075
 [6]  0.024192466  0.046877670  0.042891332 -0.039617432 -0.127540370
[11]  0.041866948  0.000000000 -0.069489050  0.047362497  0.033606639
[16]  0.042503922 -0.077735687  0.021668017  0.034959200 -0.006089641
attr(,"diag_hessian")
 [1] -0.002162576 -0.008080690  0.000000000  0.000000000 -0.004780525
 [6] -0.024212464 -0.003071418 -0.006896089 -0.035858544 -0.067821221
[11] -0.007890209  0.000000000 -0.057924744 -0.002498375 -0.015583063
[16] -0.007298861 -0.063263716 -0.022930603 -0.012776883 -0.047766527
[1] 53.46367


In [3]:
rc._loglik_sat

0.0